In [203]:
DATA_FOLDER = '../../../ADA2017-Tutorials/02 - Intro to Pandas/Data/'

## TASK 1: EBOLA DATA

In [204]:
import pandas as pd
import os.path
from calendar import monthrange
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [205]:
EBOLA_FOLDER = DATA_FOLDER+'ebola/'

guinea = EBOLA_FOLDER+'guinea_data/'
liberia = EBOLA_FOLDER+'liberia_data/'
sl = EBOLA_FOLDER+'sl_data' #Sierra Leone
        
COUNTRIES=['Guinea','Liberia','Sierra Leone']
TERMS='Newly reported deaths|Newly Reported Cases|New case|New death|new_deaths'

In [206]:
def loadAllFile(FolderPath):
    data = pd.DataFrame()
    #For each file in the country directory
    for f in os.listdir(FolderPath):
        if f.endswith(".csv"):
            file_path = os.path.join(FolderPath, f)
            file_data = pd.read_csv(file_path)
            data = data.append(file_data, ignore_index=True)
    return data

### Guinea

In [207]:
data_guinea = loadAllFile(guinea)
data_guinea.Date=pd.to_datetime(data_guinea.Date) 
data_guinea['Country']='Guinea'
data_guinea = data_guinea.set_index(['Country','Date'])

### Liberia 

In [208]:
data_liberia = loadAllFile(liberia)
data_liberia.rename(columns={'Variable': 'Description','National': 'Totals'}, inplace=True)
data_liberia.Date=pd.to_datetime(data_liberia.Date) 
data_liberia.drop('Unnamed: 18', axis=1, inplace=True)
data_liberia['Country']='Liberia'
data_liberia = data_liberia.set_index(['Country','Date'])

### Sierra Leone

In [209]:
data_sl = loadAllFile(sl)
data_sl.rename(columns={'variable': 'Description','date': 'Date','National': 'Totals'}, inplace=True)
data_sl.Date=pd.to_datetime(data_sl.Date) 
data_sl.drop('Unnamed: 18', axis=1, inplace=True)
data_sl.dropna(axis=1, how='all', inplace=True) #Drop columns where all of the elements are nan (Police traning School)
data_sl['Country']='Sierra Leone'
data_sl = data_sl.set_index(['Country','Date'])

##  All Data

In [210]:
#Selecting important column to answer the question
data_guinea = data_guinea[['Description','Totals']]
data_liberia = data_liberia[['Description','Totals']]
data_sl = data_sl[['Description','Totals']]

#Merge
data = pd.concat([data_guinea,data_liberia,data_sl])

#droping row with no information
data.dropna(axis=0, how='any', inplace=True) #Drop rows where any of the elements are nan

In [211]:
result = pd.DataFrame()
for country in COUNTRIES:
    all_dates = data.xs(country, level='Country').index.map(lambda t: str(t.year)+'-'+str('%02d' % t.month)).unique()
    for date_start in all_dates:
        date_start = date_start+'-'+"01"
        date_start
        days_in_month = monthrange(int(date_start[0:4]), int(date_start[5:7]))[1]
        date_end = date_start[0:8]+ str(days_in_month)
        
        tmp = data.xs(country, level='Country').loc[date_start:date_end]
        tmp = tmp[tmp["Description"].str.contains(TERMS, case=False)]
        tmp['Totals']= tmp['Totals'].astype(float)

        tmp = tmp.groupby('Description').sum()/days_in_month 
        tmp['Date']=date_start[0:4]+'-'+date_start[5:7]
        tmp['Country']=country
        tmp = tmp.reset_index()
        tmp = tmp.set_index(['Country','Date','Description'])
        result = result.append(tmp)
result

Totals
Country      Date    Description                                            
Guinea       2014-08 New cases of confirmed                         2.000000
                     New cases of confirmed among health workers    0.000000
                     New cases of probables                         0.258065
                     New cases of suspects                          1.903226
                     New deaths registered                          0.483871
                     New deaths registered among health workers     0.000000
                     New deaths registered today                    0.064516
                     New deaths registered today (confirmed)        0.064516
                     New deaths registered today (probables)        0.000000
                     New deaths registered today (suspects)         0.000000
                     Total new cases registered so far              4.161290
             2014-09 New cases of confirmed                         6.933333
                     New cases of confirmed among health workers    0.233333
                     New cases of probables                         0.633333
                     New cases of suspects                          2.900000
                     New deaths registered                          1.900000
                     New deaths registered among health workers     0.000000
                     Total new cases registered so far             10.466667
             2014-10 New cases of confirmed                         0.193548
                     New cases of confirmed among health workers    0.064516
                     New cases of probables                         0.000000
                     New cases of suspects                          0.903226
                     New deaths registered                          0.483871
                     New deaths registered among health workers     0.000000
                     Total new cases registered so far              1.096774
Liberia      2014-06 New Case/s (Probable)                          0.266667
                     New Case/s (Suspected)                         0.566667
                     New case/s (confirmed)                         0.500000
                     Newly Reported Cases in HCW                    0.266667
                     Newly Reported deaths in HCW                   0.033333
...                                                                      ...
             2014-08 Newly Reported Cases in HCW                    0.258065
                     Newly Reported deaths in HCW                   0.322581
                     Newly reported deaths                          6.741935
             2014-09 New Case/s (Probable)                         23.466667
                     New Case/s (Suspected)                        22.666667
                     New case/s (confirmed)                         4.933333
                     Newly Reported Cases in HCW                    1.133333
                     Newly Reported deaths in HCW                   0.500000
                     Newly reported deaths                         28.833333
             2014-10 New Case/s (Probable)                         14.322581
                     New Case/s (Suspected)                        21.322581
                     New case/s (confirmed)                         1.096774
                     Newly Reported Cases in HCW                    0.580645
                     Newly Reported deaths in HCW                   0.032258
                     Newly reported deaths                         22.612903
             2014-11 New Case/s (Probable)                          3.500000
                     New Case/s (Suspected)                         8.433333
                     New case/s (confirmed)                         1.300000
                     Newly Reported Cases in HCW                    0.166667
                     Newly Reported deaths in HCW                 